In [23]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [24]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='bab9558b-3e', project_access_token='p-c')



In [25]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',verify=True).text

In [26]:
soap = BeautifulSoup(response,'lxml')
My_table = soap.find('table')# ,{"class":"wikitable sortableit"})
tab=My_table.findAll('tr')

column_val = tab[0].findAll('th')
dfPostal = pd.DataFrame(columns=[column_val[0].text.rstrip().replace(" ", "_"),column_val[1].text.rstrip(),column_val[2].text.rstrip()])


j=0
for i in range(1, len(tab)):
    colVal=tab[i].findAll('td')
    if colVal[1].text.rstrip() != 'Not assigned':
      dfPostal.loc[j]=[colVal[0].text.rstrip(),colVal[1].text.rstrip(),colVal[2].text.rstrip().replace('/',',')]
      j=j+1

In [27]:
!pip install googlemaps

  Stored in directory: /home/dsxuser/.cache/pip/wheels/85/13/9a/6be797f54ea8759a77961114dcbac86fef2a6cc7a4d6ad9785
Successfully built googlemaps


In [29]:
import googlemaps
gmaps = googlemaps.Client(key='ORo')


In [30]:

dfPostal['Latitude'] = np.nan
dfPostal['Longitude'] = np.nan

for i in dfPostal.index:
 geocode_result = gmaps.geocode(dfPostal.loc[i]['Postal_code'])
 if len(geocode_result) > 0:
  dfPostal.ix[i,'Latitude']= float(geocode_result[0]['geometry']['location']['lat'])
  dfPostal.ix[i,'Longitude']= float(geocode_result[0]['geometry']['location']['lng'])  

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [31]:
dfPostal.head()

,Postal_code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,32.363577,-90.152413
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.715807,-79.449963
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.658448,-79.375537


In [32]:
allPostCode=pd.read_csv(project.get_file('Geospatial_Coordinates.csv'))
allPostCode.columns = allPostCode.columns.str.replace(' ', '')

In [37]:
for i in dfPostal.index:
 if np.isnan(dfPostal.loc[i]['Latitude']):
  temp=allPostCode.query('PostalCode == "'+ dfPostal.loc[i]['Postal_code'] +'"')
  if len(temp) > 0:
   dfPostal.ix[i,'Latitude']= float(temp['Latitude'])
   dfPostal.ix[i,'Longitude']= float(temp['Longitude'])  



/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [38]:
dfPostal.head()

,Postal_code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,32.363577,-90.152413
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.715807,-79.449963
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.658448,-79.375537


In [40]:
dfPostal.shape

(103, 5)

In [42]:
dfPostal.to_csv('neighborDatasheet.csv') 

In [45]:
project.save_data(file_name = "neighborDatasheet.csv",data = dfPostal.to_csv(index=False))

{'file_name': 'neighborDatasheet.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'predictcustomerinteresttooptimize-donotdelete-pr-gq3yh0rugleebo',
 'asset_id': '4a9b2e12-24f0-45d3-bf3c-4f76921ac0bd'}